In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from lib.imports import *
from lib.data import *

In [3]:
G_list = load_G_list(data_path='data/rome', index_file='data_index.txt', cache='G_list.pickle')

In [ ]:
data_list = generate_data_list(G_list, 
                               sparse=4, 
                               pivot_mode='maxmin',
                               init_mode='random',
                               edge_index='full_edge_index',
                               edge_attr='full_edge_attr',
                               pmds_list=pickle.load(open('pmds_list.pickle', 'rb')),
                               device='cpu')

preprocess G:   0%|          | 0/11531 [00:00<?, ?it/s]

In [ ]:
batch = Batch.from_data_list(data_list)

In [9]:
batch.cluster_edge_sparsity.mean()

tensor(0.3131)

In [10]:
batch.grouped_edge_sparsity.mean()

tensor(0.4522)

In [11]:
batch.sparse_edge_sparsity.mean()

tensor(0.2566)

In [12]:
batch.hierarchical_edge_sparsity.mean()

tensor(0.0943)

In [ ]:
data_list[8473]

In [7]:
batch.hierarchical_edge_sparsity.mean()

tensor(0.0233)

In [54]:
def generate_apsp(G):
    apsp_dict = dict(nx.all_pairs_shortest_path_length(G))
    return np.array([[apsp_dict[j][k] for k in sorted(apsp_dict[j].keys())] for j in sorted(apsp_dict.keys())])

In [55]:
def generate_pivots(nodes, apsp, k):
    partial_apsp = apsp[nodes, :][:, nodes]
    pivots = [partial_apsp.max(axis=1).argmax()]
    for _ in range(k - 1):
        pivots.append(partial_apsp[:, pivots].min(axis=1).argmax())
    return np.array(nodes)[pivots]

In [56]:
def get_pivot_groups(nodes, apsp, pivots):
    partial_apsp = apsp[pivots, :][:, nodes]
    groups = np.zeros_like(partial_apsp)
    groups[partial_apsp.argmin(axis=0), np.arange(groups.shape[1])] = 1
    return {p: (n := np.array(nodes)[g.astype(bool)])[n != p] for p, g in zip(pivots, groups)}

In [57]:
def get_recursive_pivot_groups(nodes, apsp, k):
    pivots = generate_pivots(nodes, apsp, k)
    groups = get_pivot_groups(nodes, apsp, pivots)
    for p in pivots:
        if len(groups[p]) > k:
            groups[p] = get_recursive_pivot_groups(groups[p], apsp, k)
    return groups

In [58]:
def get_pivot_group_cardinalities(groups):
    cardin_dict = dict()
    def get_cardinalities(groups):
        if type(groups) is not dict:
            cardin_dict.update(zip(groups, np.ones_like(groups)))
            return len(groups) + 1
        cardin = list(map(get_cardinalities, groups.values()))
        cardin_dict.update(zip(groups, cardin))
        return sum(cardin) + 1
    get_cardinalities(groups)
    return np.array(sorted(cardin_dict.items()))[:, 1]

In [64]:
def get_peer_pivot_dense_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        if type(groups) is dict:
            edges.update([(p, q) for p in groups for q in groups if p != q])
            for p in groups:
                add_all_edges(groups[p])
    add_all_edges(groups)
    return edges

def get_same_level_dense_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        edges.update([(p, q) for p in groups for q in groups if p != q])
        if type(groups) is dict:
            for p in groups:
                add_all_edges(groups[p])
    add_all_edges(groups)
    return edges

def get_adjacent_level_dense_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        if type(groups) is dict:
            descendents = np.concatenate([add_all_edges(groups[p]) for p in groups])
            edges.update([(p, i) for p in groups for i in descendents])
            return np.array(list(groups))
        else:
            return groups
    add_all_edges(groups)
    return edges

def get_cross_level_dense_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        if type(groups) is dict:
            descendents = np.concatenate([add_all_edges(groups[p]) for p in groups])
            edges.update([(p, i) for p in groups for i in descendents])
            return np.concatenate((list(groups), descendents))
        else:
            return groups
    add_all_edges(groups)
    return edges

def get_tree_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        if type(groups) is dict:
            edges.update([(p, i) for p in groups for i in groups[p]])
            for p in groups:
                add_all_edges(groups[p])
    add_all_edges(groups)
    return edges

def get_forward_edge_set(groups):
    edges = set()
    def add_all_edges(groups):
        if type(groups) is dict:
            descendents = {p: add_all_edges(groups[p]) for p in groups}
            edges.update([(p, i) for p in groups for i in descendents[p]])
            return np.concatenate((list(groups), *descendents.values()))
        else:
            return groups
    add_all_edges(groups)
    return edges

In [68]:
def get_hierarchical_cluster_edge_set(groups):
    return get_tree_edge_set(groups).union(get_same_level_dense_edge_set(groups))

def get_recursive_cluster_edge_set(groups):
    return get_forward_edge_set(groups).union(get_same_level_dense_edge_set(groups))

def get_hierarchical_sparse_edge_set(groups):
    return get_adjacent_level_dense_edge_set(groups).union(get_peer_pivot_dense_edge_set(groups))

def get_recursive_sparse_edge_set(groups):
    return get_cross_level_dense_edge_set(groups).union(get_peer_pivot_dense_edge_set(groups))

In [69]:
def create_edge_index(*edge_sets, device='cpu'):
    all_edges = reduce(lambda x, y: x.union(y), edge_sets, set())
    reverse_edges = set(map(lambda x: x[::-1], all_edges))
    sorted_symmetric_edges = sorted(list(all_edges.union(reverse_edges)))
    return torch.tensor(sorted_symmetric_edges, dtype=torch.long, device=device).t()

In [70]:
def generate_regular_edge_attr_new(edge_index, apsp, device='cpu'):
    d = apsp[edge_index[0], edge_index[1]]
    w = 1 / d ** 2
    return torch.tensor(np.stack([d, w], axis=1), dtype=torch.float, device=device)

In [71]:
def generate_pivot_edge_attr_new(edge_index, apsp, cardinalities, device='cpu'):
    d = apsp[edge_index[0], edge_index[1]]
    s = cardinalities[edge_index[0]]
    w = s / d ** 2
    return torch.tensor(np.stack([d, w], axis=1), dtype=torch.float, device=device)

In [72]:
def generate_symmetric_pivot_edge_attr(edge_index, apsp, cardinalities, device='cpu'):
    d = apsp[edge_index[0], edge_index[1]]
    s = cardinalities[edge_index[0]] * cardinalities[edge_index[1]]
    w = s / d ** 2
    return torch.tensor(np.stack([d, w], axis=1), dtype=torch.float, device=device)

In [73]:
G = G_list[0]

In [74]:
apsp = generate_apsp(G)
apsp

array([[0, 4, 2, 2, 2, 1, 1, 3, 1, 2, 3, 3, 3, 4, 2, 1],
       [4, 0, 4, 4, 4, 5, 5, 5, 5, 2, 3, 3, 1, 2, 4, 3],
       [2, 4, 0, 2, 4, 3, 1, 3, 3, 2, 3, 3, 3, 4, 2, 1],
       [2, 4, 2, 0, 2, 3, 3, 2, 1, 2, 1, 3, 3, 2, 1, 1],
       [2, 4, 4, 2, 0, 3, 3, 4, 1, 2, 3, 1, 3, 2, 3, 3],
       [1, 5, 3, 3, 3, 0, 2, 4, 2, 3, 4, 4, 4, 5, 3, 2],
       [1, 5, 1, 3, 3, 2, 0, 4, 2, 3, 4, 4, 4, 5, 3, 2],
       [3, 5, 3, 2, 4, 4, 4, 0, 3, 3, 3, 4, 4, 4, 1, 2],
       [1, 5, 3, 1, 1, 2, 2, 3, 0, 3, 2, 2, 4, 3, 2, 2],
       [2, 2, 2, 2, 2, 3, 3, 3, 3, 0, 3, 1, 1, 2, 2, 1],
       [3, 3, 3, 1, 3, 4, 4, 3, 2, 3, 0, 2, 2, 1, 2, 2],
       [3, 3, 3, 3, 1, 4, 4, 4, 2, 1, 2, 0, 2, 1, 3, 2],
       [3, 1, 3, 3, 3, 4, 4, 4, 4, 1, 2, 2, 0, 1, 3, 2],
       [4, 2, 4, 2, 2, 5, 5, 4, 3, 2, 1, 1, 1, 0, 3, 3],
       [2, 4, 2, 1, 3, 3, 3, 1, 2, 2, 2, 3, 3, 3, 0, 1],
       [1, 3, 1, 1, 3, 2, 2, 2, 2, 1, 2, 2, 2, 3, 1, 0]])

In [75]:
pivots = generate_pivots(G.nodes, apsp, 4)
pivots

array([1, 5, 7, 2])

In [76]:
groups = get_pivot_groups(G.nodes, apsp, pivots)
groups

{1: array([ 9, 10, 11, 12, 13]),
 5: array([0, 4, 8]),
 7: array([ 3, 14]),
 2: array([ 6, 15])}

In [77]:
groups = get_recursive_pivot_groups(G.nodes, apsp, 4)
groups

{1: {9: array([], dtype=int64),
  10: array([13]),
  11: array([], dtype=int64),
  12: array([], dtype=int64)},
 5: array([0, 4, 8]),
 7: array([ 3, 14]),
 2: array([ 6, 15])}

In [78]:
cardinalities = get_pivot_group_cardinalities(groups)

In [87]:
hier_eset = get_hierarchical_cluster_edge_set(groups)
print(len(hier_eset))
hier_eset

62


{(0, 0),
 (0, 4),
 (0, 8),
 (1, 1),
 (1, 2),
 (1, 5),
 (1, 7),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (2, 1),
 (2, 2),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 15),
 (3, 3),
 (3, 14),
 (4, 0),
 (4, 4),
 (4, 8),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 4),
 (5, 5),
 (5, 7),
 (5, 8),
 (6, 6),
 (6, 15),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 5),
 (7, 7),
 (7, 14),
 (8, 0),
 (8, 4),
 (8, 8),
 (9, 9),
 (9, 10),
 (9, 11),
 (9, 12),
 (10, 9),
 (10, 10),
 (10, 11),
 (10, 12),
 (10, 13),
 (11, 9),
 (11, 10),
 (11, 11),
 (11, 12),
 (12, 9),
 (12, 10),
 (12, 11),
 (12, 12),
 (13, 13),
 (14, 3),
 (14, 14),
 (15, 6),
 (15, 15)}

In [88]:
recur_eset = get_recursive_cluster_edge_set(groups)
print(len(recur_eset))
recur_eset

63


{(0, 0),
 (0, 4),
 (0, 8),
 (1, 1),
 (1, 2),
 (1, 5),
 (1, 7),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (2, 1),
 (2, 2),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 15),
 (3, 3),
 (3, 14),
 (4, 0),
 (4, 4),
 (4, 8),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 4),
 (5, 5),
 (5, 7),
 (5, 8),
 (6, 6),
 (6, 15),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 5),
 (7, 7),
 (7, 14),
 (8, 0),
 (8, 4),
 (8, 8),
 (9, 9),
 (9, 10),
 (9, 11),
 (9, 12),
 (10, 9),
 (10, 10),
 (10, 11),
 (10, 12),
 (10, 13),
 (11, 9),
 (11, 10),
 (11, 11),
 (11, 12),
 (12, 9),
 (12, 10),
 (12, 11),
 (12, 12),
 (13, 13),
 (14, 3),
 (14, 14),
 (15, 6),
 (15, 15)}

In [81]:
hier_eset = get_hierarchical_sparse_edge_set(groups)
print(len(hier_eset))
hier_eset

80


{(1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 14),
 (1, 15),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 14),
 (2, 15),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (5, 10),
 (5, 11),
 (5, 12),
 (5, 14),
 (5, 15),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (7, 12),
 (7, 14),
 (7, 15),
 (9, 9),
 (9, 10),
 (9, 11),
 (9, 12),
 (9, 13),
 (10, 9),
 (10, 10),
 (10, 11),
 (10, 12),
 (10, 13),
 (11, 9),
 (11, 10),
 (11, 11),
 (11, 12),
 (11, 13),
 (12, 9),
 (12, 10),
 (12, 11),
 (12, 12),
 (12, 13)}

In [82]:
recur_eset = get_recursive_sparse_edge_set(groups)
print(len(recur_eset))
recur_eset

84


{(1, 0),
 (1, 1),
 (1, 2),
 (1, 3),
 (1, 4),
 (1, 5),
 (1, 6),
 (1, 7),
 (1, 8),
 (1, 9),
 (1, 10),
 (1, 11),
 (1, 12),
 (1, 13),
 (1, 14),
 (1, 15),
 (2, 0),
 (2, 1),
 (2, 2),
 (2, 3),
 (2, 4),
 (2, 5),
 (2, 6),
 (2, 7),
 (2, 8),
 (2, 9),
 (2, 10),
 (2, 11),
 (2, 12),
 (2, 13),
 (2, 14),
 (2, 15),
 (5, 0),
 (5, 1),
 (5, 2),
 (5, 3),
 (5, 4),
 (5, 5),
 (5, 6),
 (5, 7),
 (5, 8),
 (5, 9),
 (5, 10),
 (5, 11),
 (5, 12),
 (5, 13),
 (5, 14),
 (5, 15),
 (7, 0),
 (7, 1),
 (7, 2),
 (7, 3),
 (7, 4),
 (7, 5),
 (7, 6),
 (7, 7),
 (7, 8),
 (7, 9),
 (7, 10),
 (7, 11),
 (7, 12),
 (7, 13),
 (7, 14),
 (7, 15),
 (9, 9),
 (9, 10),
 (9, 11),
 (9, 12),
 (9, 13),
 (10, 9),
 (10, 10),
 (10, 11),
 (10, 12),
 (10, 13),
 (11, 9),
 (11, 10),
 (11, 11),
 (11, 12),
 (11, 13),
 (12, 9),
 (12, 10),
 (12, 11),
 (12, 12),
 (12, 13)}

In [83]:
edge_index = create_edge_index(G.edges, hier_eset)
edge_index

tensor([[ 0,  0,  0,  0,  0,  0,  0,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,  1,
          1,  1,  1,  1,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
          2,  3,  3,  3,  3,  3,  3,  3,  3,  4,  4,  4,  4,  4,  4,  5,  5,  5,
          5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  5,  6,  6,  6,  6,  6,  7,
          7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  7,  8,  8,  8,  8,
          8,  8,  8,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 10, 10, 10, 10, 10,
         10, 10, 10, 10, 10, 11, 11, 11, 11, 11, 11, 11, 11, 11, 11, 12, 12, 12,
         12, 12, 12, 12, 12, 12, 13, 13, 13, 13, 14, 14, 14, 14, 14, 14, 15, 15,
         15, 15, 15, 15, 15, 15],
        [ 1,  2,  5,  6,  7,  8, 15,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10,
         11, 12, 14, 15,  0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14,
         15,  1,  2,  5,  7,  8, 10, 14, 15,  1,  2,  5,  7,  8, 11,  0,  1,  2,
          3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 14, 15,  0,  1,  2,  5,  7

In [70]:
generate_symmetric_pivot_edge_attr(edge_index, apsp, cardinalities)

tensor([[1.0000e+00, 5.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [2.0000e+00, 1.0000e+00],
        [3.0000e+00, 4.4444e-01],
        [7.0000e+00, 1.7143e+00],
        [1.0000e+00, 4.0000e+00],
        [1.0000e+00, 4.0000e+00],
        [2.0000e+00, 1.2500e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [1.0000e+00, 5.0000e+00],
        [2.0000e+00, 1.2500e+00],
        [1.0000e+00, 2.0000e+01],
        [5.0000e+00, 4.8000e+00],
        [1.0000e+00, 5.0000e+00],
        [2.0000e+00, 1.2500e+00],
        [1.0000e+00, 4.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [2.0000e+00, 1.0000e+00],
        [1.0000e+00, 1.0000e+00],
        [2.0000e+00, 1.2500e+00],
        [3.0000e+00, 5.5556e-01],
        [3.0000e+00, 5.5556e-01],
        [1.0000e+00, 1.2000e+02],
        [1.000

In [37]:
[(i, j) for i in range(5) for j in range(i)]

[(1, 0),
 (2, 0),
 (2, 1),
 (3, 0),
 (3, 1),
 (3, 2),
 (4, 0),
 (4, 1),
 (4, 2),
 (4, 3)]

In [51]:
np.array([[1,2,3],[4,5,6],[7,8,9]])[[0,2], :][:, [0,2]]

array([[1, 3],
       [7, 9]])

In [77]:
np.append(a, 1)

array([1.])

In [21]:
np.argmax(list(map(nx.classes.graph.Graph.number_of_nodes, G_list)))

8473

In [17]:
type(G)

networkx.classes.graph.Graph